In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import os 
import seaborn as sns
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler,  RobustScaler
import pickle
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from keras import regularizers, layers, optimizers, initializers
from tensorflow.keras.callbacks import TensorBoard
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error

from tensorflow.keras.applications import EfficientNetV2M, ConvNeXtBase
import numpy as np
import gc
import tensorflow_addons as tfa
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Input, Concatenate, Dropout, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
print(f'Current GPU allocator: {os.getenv("TF_GPU_ALLOCATOR")}')

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            print(f'Setting memory growth for {gpu}')
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
    

In [ ]:
study_name = '426_convnextbase_003_998_1'

In [ ]:

mean_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

In [ ]:
pd.set_option('display.max_rows', None)

# Aseta näyttämään rajoittamaton määrä sarakkeita
pd.set_option('display.max_columns', None)

In [ ]:

pickle_file_path = './data/test_convnextbase_df_003_998.pickle'

with open(pickle_file_path, 'rb') as f:
    test_df = pickle.load(f)

pickle_file_path = './data/train_convnextbase_df_003_998.pickle'

with open(pickle_file_path, 'rb') as f:
    train_df = pickle.load(f)

In [ ]:
# # FOR TESTING IMAGE AUGEMENTATION
# train_df = train_df.sample(1000)
# test_df = test_df.sample(1000)

In [ ]:
feat = pd.read_csv('./data/test.csv')
FEATURE_COLS = feat.columns[1:].tolist()

In [ ]:
# train_images_path = './data/train_images/'
# test_images_path = './data/test_images/'    

# train_df['image_path'] = train_df['id'].apply(lambda x: os.path.join(train_images_path, f'{x}.jpeg'))
# test_df['image_path'] = test_df['id'].apply(lambda x: os.path.join(test_images_path, f'{x}.jpeg'))

In [ ]:
# for column in mean_columns:
#     lower_quantile = train_df[column].quantile(0.005)
#     upper_quantile = train_df[column].quantile(0.985)  
#     train_df = train_df[(train_df[column] >= lower_quantile) & (train_df[column] <= upper_quantile)]

In [ ]:
# from sklearn.model_selection import StratifiedKFold

# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# for i, trait in enumerate(mean_columns):

#     # Determine the bin edges dynamically based on the distribution of traits
#     bin_edges = np.percentile(train_df[trait], np.linspace(0, 100, 5 + 1))
#     train_df[f"bin_{i}"] = np.digitize(train_df[trait], bin_edges)

# # Concatenate the bins into a final bin
# train_df["final_bin"] = (
#     train_df[[f"bin_{i}" for i in range(len(mean_columns))]]
#     .astype(str)
#     .agg("".join, axis=1)
# )

# # Perform the stratified split using final bin
# train_df = train_df.reset_index(drop=True)
# for fold, (train_idx, valid_idx) in enumerate(skf.split(train_df, train_df["final_bin"])):
#     train_df.loc[valid_idx, "fold"] = fold

In [ ]:
scaler_feat = RobustScaler()

train_original = train_df.copy()
train_plot = train_df.copy()
sample_df = train_df.copy()

train_df = sample_df[sample_df.fold != 1]
valid_df = sample_df[sample_df.fold == 1]

print(f"# Num Train: {len(train_df)} | Num Valid: {len(valid_df)}")


In [ ]:
import glob

directory_path = './NN_search'
pattern = f"{directory_path}/{study_name}*.h5"

files = glob.glob(pattern)

max_r2_score = float('-inf')
best_model = None

# Käy läpi jokainen tiedosto ja etsi suurin r2_score_inv
for file in files:
    value = float(file.split('best_val')[1].split('_')[1])
    if value > max_r2_score:
        max_r2_score = value
        best_model = file


# Tulosta suurin löydetty r2_score_inv ja vastaava tiedosto
print(f"Best R2-score: {max_r2_score:.5f}")
if best_model:
    print(f"Best model: {best_model}")
else:
    print("No best model found")

best_log_transforms_name =  f'./NN_search/{study_name}_{max_r2_score:.5f}_best_log_transforms.pickle'
best_scalers_name = f'./NN_search/{study_name}_{max_r2_score:.5f}_best_scalers.pickle'


print(f'Opening scalers from {best_scalers_name}')
with open(best_scalers_name, 'rb') as f:
    scaler_transforms = pickle.load(f)

log_transforms = {'X4_mean': 10, 'X11_mean': 10, 'X18_mean': 10, 'X50_mean': 4, 'X26_mean': 7, 'X3112_mean': 2}

        

In [ ]:
print(f'Scaler are: {scaler_transforms}')
print(f'Log transforms are: {log_transforms}')

In [ ]:
def augment_image(img):

    img = img / 255.0
    
    img = tf.image.random_flip_left_right(img)
    
    # img = tf.image.rot90(img, k=tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32))

    img = tf.image.random_brightness(img, max_delta=0.1)
    img = tf.image.random_hue(img, max_delta=0.1)
    img = tf.image.random_saturation(img, lower=0.9, upper=1.1)
    img = tf.image.random_contrast(img, lower=0.9, upper=1.1)

    img = tf.image.random_jpeg_quality(img, min_jpeg_quality=85, max_jpeg_quality=100)

    # angle = tf.random.uniform([], minval=-np.pi/8, maxval=np.pi/8, dtype=tf.float32)
    # img = tfa.image.rotate(img, angles=angle)

    crop_size = tf.random.uniform(shape=[], minval=420, maxval=480, dtype=tf.int32)
    img = tf.image.random_crop(img, size=[crop_size, crop_size, 3])
    img = tf.image.resize(img, [480, 480]) 


    
    
    theta = tf.random.uniform([], minval=-0.2, maxval=0.2)
    tx = tf.random.uniform([], minval=-45, maxval=45)
    ty = tf.random.uniform([], minval=-45, maxval=45)
    cos_theta = tf.cos(theta)
    sin_theta = tf.sin(theta)
    
    transformation_matrix = [cos_theta, -sin_theta, tx,
                                sin_theta, cos_theta, ty,
                                0, 0]

    img = tfa.image.transform(img, transformation_matrix, interpolation="BILINEAR")
    
    img = img * 255.0

    return img


def process_image(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    img = augment_image(img)  
    # img = tf.cast(img, tf.uint8)    

    # tf.print("Final min and max in process_images:", tf.reduce_min(img), tf.reduce_max(img))
    # tf.print("Image type: ", img.dtype)

    return img

def process_image_valid(file_path):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (480, 480))
    # tf.print("Final min and max in process_image_valid:", tf.reduce_min(img), tf.reduce_max(img))
    # img = tf.cast(img, tf.uint8)
    return img

# Define your dataset processing function
def process_path_train(file_path, targets):
    img = process_image(file_path)
    return img, targets


def process_path_valid(file_path, targets):
    img = process_image_valid(file_path)
    return img, targets

def process_path_test(file_path, dummy):
    img = process_image_valid(file_path)    
    return img, dummy


In [ ]:
y_train = train_df[mean_columns]
y_valid = valid_df[mean_columns]


y_train_transformed = y_train.copy()
y_valid_transformed = y_valid.copy()

for target, log_base in log_transforms.items():

    if log_base is not None and log_base != 'sqrt' and log_base != 'cbrt':
        y_train_transformed[target] = np.log(y_train[target]) / np.log(log_base)
        y_valid_transformed[target] = np.log(y_valid[target]) / np.log(log_base)

    elif log_base == 'sqrt':
        y_train_transformed[target] = np.sqrt(y_train[target])
        y_valid_transformed[target] = np.sqrt(y_valid[target])

    elif log_base == 'cbrt':
        y_train_transformed[target] = np.cbrt(y_train[target])
        y_valid_transformed[target] = np.cbrt(y_valid[target])

    else:
        y_train_transformed[target] = y_train[target]
        y_valid_transformed[target] = y_valid[target]    


y_train_transformed = scaler_transforms.fit_transform(y_train_transformed)
y_valid_transformed = scaler_transforms.transform(y_valid_transformed)



In [ ]:
BUFFER_SIZE = 55000 
EPOCHS = 6
BATCH_SIZE = 16

train_images_path = train_df['image_path'].values
valid_images_path = valid_df['image_path'].values

train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path, y_train_transformed))
train_dataset = train_dataset.shuffle(BUFFER_SIZE)

valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images_path, y_valid_transformed))

train_dataset = train_dataset.map(process_path_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

len_train = len(train_dataset) * EPOCHS

print(f'LR schedule steps: {len_train}')




In [ ]:
tf.keras.backend.clear_session()
gc.collect()


def r2_score_tf(y_true, y_pred):

    try: 
        ss_res = tf.reduce_sum(tf.square(y_true - y_pred), axis=0)
        ss_tot = tf.reduce_sum(tf.square(y_true - tf.reduce_mean(y_true, axis=0)), axis=0)
        r2 = 1 - ss_res/(ss_tot + tf.keras.backend.epsilon())
        r2 = tf.where(tf.math.is_nan(r2), tf.zeros_like(r2), r2) 
        return tf.reduce_mean(tf.maximum(r2, 0.0))
    except Exception as e:
        # print(f'Error in r2_score_tf: {e}')
        return float('-inf')



image_input_avg = Input(shape=(480, 480, 3), name='image_input_avg')

eff_avg_base =  ConvNeXtBase(weights='imagenet', include_top=False, pooling='avg', input_tensor=image_input_avg)

eff_avg_base.trainable = True
for layer in eff_avg_base.layers[:-100]:
    layer.trainable = False

eff_avg_base = Dropout(0.5)(eff_avg_base.output)

# eff_drouput = Dropout(0.5)(eff_avg_base.output)

first_layer = Dense(1202, activation='sigmoid', name='first_layer', kernel_initializer='random_uniform')(eff_avg_base)
first_drouput = Dropout(0.611)(first_layer)

second_layer = Dense(626, activation='tanh', name='second_layer', kernel_initializer='random_uniform')(first_drouput)
second_norm = tf.keras.layers.LayerNormalization()(second_layer)
second_dropout = Dropout(0.0268)(second_norm)

third_layer = Dense(402, activation='selu', name='third_layer', kernel_initializer='he_uniform')(second_dropout)
third_norm = tf.keras.layers.LayerNormalization()(third_layer)
third_dropout = Dropout(0.008)(third_norm)

fourth_layer = Dense(241, activation='tanh', name='fourth_layer', kernel_initializer='lecun_normal')(third_dropout)
fourth_norm = tf.keras.layers.LayerNormalization()(fourth_layer)
fourth_dropout = Dropout(0.0034)(fourth_norm)

model_out = Dense(6, activation='linear', name='final_tune_output')(fourth_dropout)

finetune_model = Model(inputs=image_input_avg, outputs=model_out, name='finetune_model')

lr_schedule = tf.keras.optimizers.schedules.CosineDecay(
    1e-4,    
    alpha=0.01,
    name="CosineDecay",
    decay_steps=len_train
)

# Aseta oppimisnopeuden aikataulu
finetune_model.compile(optimizer=optimizers.Adam(learning_rate=lr_schedule), loss='mse', metrics=['mse', 'mae', 'mape', r2_score_tf])


In [ ]:
trainable_count = sum([tf.size(v).numpy() for v in finetune_model.trainable_weights])
non_trainable_count = sum([tf.size(v).numpy() for v in finetune_model.non_trainable_weights])
print(f"Total parameters: {trainable_count + non_trainable_count:,}")
print(f"Trainable parameters: {trainable_count:,}")
print(f"Non-trainable parameters: {non_trainable_count:,}")


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import Callback
import matplotlib.pyplot as plt
import io

class TrainImageLoggingCallback(Callback):
    def __init__(self, log_dir, data):
        super(TrainImageLoggingCallback, self).__init__()
        self.log_dir = log_dir
        self.data = data
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        # Fetch a batch of images
        for imgs, tar in self.data.take(1):  # Adjust depending on your dataset structure
            
            # augmented_images = tf.map_fn(augment_image, imgs)
            augmented_images = tf.cast(imgs, tf.uint8)    
        
            # Prepare the image to write to TensorBoard
            with self.writer.as_default():
                tf.summary.image("Augmented Images", augmented_images, step=epoch, max_outputs=20)

            self.writer.flush()


In [ ]:

log_folder = f"./logs/all/trial_{study_name}_429" 

print(f'Logging tensorboard to {log_folder}')
os.makedirs(log_folder, exist_ok=True)

# Aseta logitiedostojen hakemisto
tensorboard_callback = TensorBoard(log_dir=log_folder, histogram_freq=1, update_freq='batch')

In [ ]:


callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=f'./NN_search/testifinetus_{study_name}.h5', monitor='val_mse', save_best_only=True, save_weights_only=True, mode = 'min',  verbose = 1),
    tensorboard_callback,
    TrainImageLoggingCallback(log_folder, train_dataset)    
]

history = finetune_model.fit(train_dataset, validation_data=valid_dataset, epochs=EPOCHS, verbose=1, callbacks=callbacks)




In [ ]:
finetune_model.load_weights(f'./NN_search/testifinetus_{study_name}.h5')
# finetune_model.save(f'./NN_search/koe', save_format='tf') # TODO tässä ongelmaa vielä, mutta ei väliä. Malli on jo olemassa ja sitä voi käyttää. 


In [ ]:
for target, log_base in log_transforms.items():
    if log_base is not None and log_base != 'sqrt' and log_base != 'cbrt':
        train_plot[target] = np.log(train_plot[target]) / np.log(log_base)
        
    elif log_base == 'sqrt':
        train_plot[target] = np.sqrt(train_plot[target])
        
    elif log_base == 'cbrt':
        train_plot[target] = np.cbrt(train_plot[target])
        
    else:
        train_plot[target] = train_plot[target]
        
train_plot[mean_columns] = scaler_transforms.transform(train_plot[mean_columns])

In [ ]:
print(log_transforms)
print(scaler_transforms)

In [ ]:
train_original[mean_columns].describe()

In [ ]:
train_plot[mean_columns].describe()

In [ ]:


train_images_path = train_df['image_path'].values
valid_images_path = valid_df['image_path'].values

train_dataset = tf.data.Dataset.from_tensor_slices((train_images_path,  y_train_transformed))
valid_dataset = tf.data.Dataset.from_tensor_slices((valid_images_path, y_valid_transformed))

train_dataset = train_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(process_path_valid, num_parallel_calls=tf.data.experimental.AUTOTUNE)

BATCH_SIZE = 32

train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.batch(BATCH_SIZE).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)





In [ ]:

## TRAINING DATA TEST

tf.keras.backend.clear_session()
gc.collect()

train_pred = finetune_model.predict(train_dataset, verbose=1)

train_pred = scaler_transforms.inverse_transform(train_pred)

for i, target in enumerate(mean_columns):
    print(f'Logpot transforming target : {target}, log transform : {log_transforms[target]}')
    log_base = log_transforms[target]
    if log_base is not None and log_base != 'sqrt' and log_base != 'cbrt':
        train_pred[:, i] = np.power(log_base, train_pred[:, i])
    elif log_base == 'sqrt':
        train_pred[:, i] = np.square(train_pred[:, i])
    elif log_base == 'cbrt':
        train_pred[:, i] = np.power(train_pred[:, i], 3)

R2_train = r2_score(y_train, train_pred)
MSE_train = mean_squared_error(y_train, train_pred)
MAE_train = mean_absolute_error(y_train, train_pred)
MAPE_train = mean_absolute_percentage_error(y_train, train_pred)

print(f'Train scores:\nR2 : {R2_train:.5f}, MSE : {MSE_train:.5f}, MAE : {MAE_train:.5f}, MAPE : {MAPE_train:.5f}')


In [ ]:

## VALIDATION DATA TEST

tf.keras.backend.clear_session()
gc.collect()

valid_pred = finetune_model.predict(valid_dataset, verbose=1)

valid_pred = scaler_transforms.inverse_transform(valid_pred)


for i, target in enumerate(mean_columns):
    log_base = log_transforms[target]
    if log_base is not None and log_base != 'sqrt' and log_base != 'cbrt':
        valid_pred[:, i] = np.power(log_base, valid_pred[:, i])
    elif log_base == 'sqrt':
        valid_pred[:, i] = np.square(valid_pred[:, i])
    elif log_base == 'cbrt':
        valid_pred[:, i] = np.power(valid_pred[:, i], 3)

R2_valid = r2_score(y_valid, valid_pred)
MSE_valid = mean_squared_error(y_valid, valid_pred)
MAE_valid = mean_absolute_error(y_valid, valid_pred)
MAPE_valid = mean_absolute_percentage_error(y_valid, valid_pred)

print(f'Valid scores:\nR2 : {R2_valid:.5f}, MSE : {MSE_valid:.5f}, MAE : {MAE_valid:.5f}, MAPE : {MAPE_valid:.5f}')




In [ ]:
# TEST DATA 

tf.keras.backend.clear_session()
gc.collect()

test_df_copy = test_df.copy()
submission_df = test_df_copy[['id']].copy()

test_images_path = test_df_copy['image_path'].values

dummy_y = np.zeros((len(test_df_copy), 6))

test_dataset = tf.data.Dataset.from_tensor_slices((test_images_path, dummy_y))
test_dataset = test_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

predictions = finetune_model.predict(test_dataset, verbose=1)

predictions = scaler_transforms.inverse_transform(predictions)


for i, target in enumerate(mean_columns):
    print(f'Logpot transforming target: : {target}, log transform : {log_transforms[target]}')
    log_base = log_transforms[target]
    if log_base is not None and log_base != 'sqrt' and log_base != 'cbrt':
        predictions[:, i] = np.power(log_base, predictions[:, i])
    elif log_base == 'sqrt':
        predictions[:, i] = np.square(predictions[:, i])
    elif log_base == 'cbrt':
        predictions[:, i] = np.power(predictions[:, i], 3)




In [ ]:
print(predictions.shape)

In [ ]:

target_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

submission_df[target_columns] = predictions


In [ ]:
submission_df.head()

In [ ]:
print(submission_df.info())

submission_df.to_csv('./data/submission.csv', index=False)

In [ ]:
feature_model_output = finetune_model.get_layer('fourth_layer')
feature_model = Model(inputs=finetune_model.input, outputs=feature_model_output.output)


In [ ]:
train_all_paths = train_original['image_path'].values
dummy_y = np.zeros((len(train_original), 6))

train_all_dataset = tf.data.Dataset.from_tensor_slices((train_all_paths, dummy_y))
train_all_dataset = train_all_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_all_dataset = train_all_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

train_features = feature_model.predict(train_all_dataset, verbose=1)

train_original[f'model_features_{study_name}'] = train_features.tolist()

In [ ]:
train_original.head()

In [ ]:
pickle_file_path = f'./data/train_{study_name}.pickle'

print(f'Saving train_df to {pickle_file_path}')
with open(pickle_file_path, 'wb') as f:
    pickle.dump(train_original, f)

In [ ]:
test_all_paths = test_df['image_path'].values
dummy_y = np.zeros((len(test_df), 6))

test_all_dataset = tf.data.Dataset.from_tensor_slices((test_all_paths, dummy_y))
test_all_dataset = test_all_dataset.map(process_path_test, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test_all_dataset = test_all_dataset.batch(32).prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_features = feature_model.predict(test_all_dataset, verbose=1)

test_df[f'model_features_{study_name}'] = test_features.tolist()

In [ ]:
print(test_df[f'model_features_{study_name}'].head())

In [ ]:
test_df.head()

In [ ]:
pickle_file_path = f'./data/test_{study_name}.pickle'

print(f'Saving test_df to {pickle_file_path}')
with open(pickle_file_path, 'wb') as f:
    pickle.dump(test_df, f)
